In [1]:
from mattspy import BNLCondorParallel
import glob
from des_y6utils.mdet import make_mdet_cuts
from esutil.pbar import PBar
import joblib
import os
import numpy as np
import fitsio
import joblib

In [15]:
def _read_and_count(fname):
    d = fitsio.read(fname)
    msk = (
        (d['psfrec_flags'] == 0) &
        (d['gauss_flags'] == 0) &
        (d['gauss_s2n'] > 5) &
        (d['pgauss_T_flags'] == 0) &
        (d['pgauss_s2n'] > 5) &
        (d['pgauss_band_flux_flags_g'] == 0) &
        (d['pgauss_band_flux_flags_r'] == 0) &
        (d['pgauss_band_flux_flags_i'] == 0) &
        (d['pgauss_band_flux_flags_z'] == 0) &
        (d['mask_flags'] == 0) &
        (d['shear_bands'] == '123') &
        (make_mdet_cuts(d, "3")) & 
        (d["mdet_step"] == "noshear")
    )
    return np.sum(msk)

In [11]:
fnames = glob.glob("blinded_data/*.fits")

In [12]:
fnames_jk = glob.glob(
        "/gpfs02/astro/desdata/esheldon/lensing/"
        "des-lensing/y6patches/patches/*.fits"
    )

In [13]:
def _count_fnames(fnames, odir, use_joblib=False):
    tot = 0
    nres = 0
    if use_joblib:
        with joblib.Parallel(n_jobs=5, verbose=100) as par:
            tot = np.sum(par([
                joblib.delayed(_read_and_count)(os.path.abspath(fname))
                for fname in fnames
            ]))
            nres = len(fnames)
    else:
        with BNLCondorParallel(mem=4) as exc:
            futs = [
                joblib.delayed(_read_and_count)(os.path.abspath(fname))
                for fname in fnames 
                if not os.path.exists(os.path.join(odir, os.path.basename(fname) + ".count"))
            ]

            for fut in PBar(exc(futs), total=len(futs)):
                tot += fut.result()
                nres += 1

                if (nres-1) % 10 == 0:
                    print(
                        "%0.3f million (exact: %d, estimate %0.2f million)" % (
                            tot/1e6, 
                            tot, 
                            tot/1e6/nres*len(fnames)
                        ), 
                        flush=True,
                    )

    print(
        "%0.3f million (exact: %d, estimate %0.2f million)" % (
            tot/1e6, 
            tot, 
            tot/1e6/nres*len(fnames)
        ), 
        flush=True,
    )

In [17]:
_count_fnames(fnames, use_joblib=False)

|--------------------| 0/10169   0% [elapsed: 00:00 left: ?]

0.017 million (exact: 17069, estimate 173.57 million)


|--------------------| 5/10169   0% [elapsed: 01:56 left: 65:44:17] 

0.147 million (exact: 147108, estimate 135.99 million)


|--------------------| 20/10169   0% [elapsed: 02:21 left: 19:52:30]

...

148.575 million (exact: 148574880, estimate 148.84 million)


|###################-| 10157/10169  99% [elapsed: 33:53 left: 00:02]

148.730 million (exact: 148729785, estimate 148.85 million)


|###################-| 10161/10169  99% [elapsed: 40:50 left: 00:01]

RuntimeError: Condor job 44ba91d0b9da4f04a1d863b67989dd0c: status 'Removed' w/ no output

In [9]:
_count_fnames(fnames_jk, use_joblib=False)

|--------------------| 0/200   0% [elapsed: 00:00 left: ?]

0.641 million (exact: 641400, estimate 128.28 million)


|#-------------------| 10/200   5% [elapsed: 03:22 left: 1:04:07]

7.049 million (exact: 7048880, estimate 128.16 million)


|##------------------| 20/200  10% [elapsed: 03:33 left: 32:01]  

...

134.914 million (exact: 134913970, estimate 149.08 million)


|###################-| 190/200  95% [elapsed: 09:28 left: 00:29]

142.343 million (exact: 142343032, estimate 149.05 million)


|####################| 200/200 100% [elapsed: 17:39 left: 00:00]

148.852 million (exact: 148852455, estimate 148.85 million)
